In [ ]:
using Plots
using DSP
using FFTW

In [ ]:
# # pulse_duration = 0.0005  #seconds
# starting_freq = 350e3     #hz
# ending_freq = 50e3     #hz

# freqs = LinRange(starting_freq, ending_freq, Int(samp_rate * pulse_duration));
# t_pulse = LinRange(0,pulse_duration,Int(samp_rate*pulse_duration));

In [ ]:
samp_rate = 2e6  #hz
prf = 10  #Hz
τ = 0.05;
β = 250e3; #Pulse Bandwidth

In [ ]:
prt = 1/prf;
t = 0:1/samp_rate:prt;

In [ ]:
#Envelope Definition
envelope_type = "Rectangular"

if envelope_type == "Rectangular"
    a = ones(length(t));
    if(length(a) > τ * samp_rate)
        a[Int(τ*samp_rate):end] .= 0;
    end
elseif envelope_type == "Gaussian"
    a = exp.((-t.^2)/(τ^2)) #Gaussian
end


plot(t, a, title="Target Power Envelope")

In [ ]:
pulse_type = "NOrderPoly"

if pulse_type == "Increasing"
    pulse = a.*exp.(im*π*(β/τ)*t.^2);
elseif pulse_type == "Decreasing"
    pulse = a.*exp.(-im*π*β/τ*(t.^2 - 2*τ.*t));
elseif pulse_type == "Exponential"
    k = (f1/f0)^(1/τ);
    phi0 = π/2;
    phi_t = phi0 .+ 2*π*f0*(((k.^t).-1)/log(k));

    pulse_i = a.*sin.(phi_t);
    pulse_r = a.*cos.(phi_t);
    pulse = pulse_r .+ im*pulse_i;
# elseif pulse_type == "2TermNOrderPoly"
#     order = 2;
#     k = (f1 - f0)/(τ^order);
#     phi0 = π/2;
#     phi_t = phi0 .+ 2*π*(((k.*t.^(order+1))/(order+1)) .+ f0.*t);

#     pulse_i = a.*sin.(phi_t);
#     pulse_r = a.*cos.(phi_t);
#     pulse = pulse_r .+ im*pulse_i;
elseif pulse_type == "NOrderPoly"
    order = 2;
    k = ones(order);
    pushfirst!(k, 1) # To be set to f0 later
    pwr = Array((0:order));
    phi_temp = Array{Float64, 2}(undef, length(t), order+1);
    
    for i in 1:order
        for j in 1:length(t)
            phi_temp[j,i] = (k[i]*t[j]^(pwr[i]+1))/(pwr[i]+1);
        end
    end

    phi0 = π/2; # phi0 to be defined

    phi_t = phi0 .+ 2*π.*sum(phi_temp, dims=2)

    pulse_i = a.*sin.(phi_t);
    pulse_r = a.*cos.(phi_t);
    pulse = pulse_r .+ im*pulse_i
    # pulse = pulse'
end

In [ ]:
plot(t, abs.(pulse), xguide="Time", yguide="Maginitude", title="Pulse Power Envelope")

In [ ]:
plot(t,real(pulse))

In [ ]:
plot(LinRange(-samp_rate/2, samp_rate/2, length(pulse)), real(fftshift(fft(pulse))))

In [ ]:
fft_size = 512;
spec = spectrogram((pulse), fft_size; fs=samp_rate);
heatmap(spec.time, LinRange(0,samp_rate/2, Int(fft_size/2)), spec.power[begin:Int(fft_size/2) , :], xguide="Time (Seconds)", yguide="Frequency (hz)", title="Pulse Spectrogram")

In [ ]:
rx_pulse = pulse * 0.25;
pulse_compression = conv(rx_pulse, reverse(pulse));

In [ ]:
plot(abs.(pulse_compression))

In [ ]:
Threads.nthreads()